In [1]:
import pyspark
import requests
import datetime

from pyspark.sql import SparkSession 

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()



In [3]:
noise = 100
hashed_column = 'id'

request = requests.get(f'http://cluster-a:9200//take_data/{hashed_column}/{noise}')
url_content = request.content
csv_file = open("/opt/workspace/a_download.csv", 'wb')
csv_file.write(url_content)
csv_file.close()

request = requests.get(f'http://cluster-b:9300//take_data/{hashed_column}/{noise}')
url_content = request.content
csv_file = open("/opt/workspace/b_download.csv", 'wb')
csv_file.write(url_content)
csv_file.close()

df_1 = spark.read.csv(path="/opt/workspace/a_download.csv", sep=",", header=True)
df_2 = spark.read.csv(path="/opt/workspace/b_download.csv", sep=",", header=True)

df_1.createOrReplaceTempView("a_cluster_data")
df_2.createOrReplaceTempView("b_cluster_data")


In [15]:

start = datetime.datetime.now()

sql_result = spark.sql("  SELECT a_cluster_data.id as a_id, b_cluster_data.id as b_id\
                          FROM a_cluster_data, b_cluster_data\
                          WHERE a_cluster_data.name == b_cluster_data.name\
                          ")

sql_result.show()


sql_result.select('a_id').drop_duplicates().repartition(1).write.mode('overwrite').csv("/opt/workspace//id_df_a")
sql_result.select('b_id').drop_duplicates().repartition(1).write.mode('overwrite').csv("/opt/workspace//id_df_b")

stop = datetime.datetime.now()

print(stop - start)

+----------+----------+
|      a_id|      b_id|
+----------+----------+
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|  AA167396|  AA167396|
|  BY527625|  AA167396|
|  AA118930|  AA167396|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|  AA135496|  AA135496|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|Fake Index|Fake Index|
|  AA131831|  AA131831|
+----------+----------+
only showing top 20 rows

0:00:03.287744


In [16]:
sql_result.count()

239

In [12]:
df_2.show()

+----------+--------------------+--------------------+
|        id|             surname|                name|
+----------+--------------------+--------------------+
|Fake Index|e31fbd99c78e1d589...|002685f817c7450b5...|
|  AA157890|056470f11aa5e62bd...|0032999af05e220fd...|
|Fake Index|24bb4a7cef0e5441b...|003921609eb3e8b9c...|
|Fake Index|f2ef5fcb65d0b45d2...|0049067f1d87ec6d5...|
|  AA111206|d1d15dad3098fc9e0...|0049aa51a630b7912...|
|Fake Index|c64218282b8714d05...|004f97eea02372797...|
|Fake Index|6cc270fe46895b777...|00540855f878e9074...|
|  AA175050|7207ebf10d0512bcc...|0059209c6631b5cef...|
|Fake Index|ab415813ac0656f51...|008e0749e654452f5...|
|    AA2923|d094b4855e6610eb2...|009eb334164830b54...|
|Fake Index|19b0c71007a575222...|00b0d64bb46f34f34...|
|Fake Index|cc52227b3ff6b39e4...|00b3ceaecf48c4360...|
|Fake Index|ac766d46911339678...|00be3dd19f080c425...|
|Fake Index|b1a329625e507f38f...|00be3dd19f080c425...|
|Fake Index|76e0eae85a5d516c0...|00ce5de462b556c13...|
|   AA1818

In [44]:
myfiles = {'file': open('/opt/workspace/id_df_a/part-00000-2cc8bcdd-f1bf-4411-9a7d-50b32be8788b-c000.csv' ,'rb')}
requests.post(f'http://cluster-a:9200//take_data/', data={'column': hashed_column}, files = myfiles)

myfiles = {'file': open('/opt/workspace/id_df_b/part-00000-eb201cb5-f83a-4019-8c79-014226656171-c000.csv' ,'rb')}
requests.post(f'http://cluster-b:9300//take_data/', data={'column': hashed_column}, files = myfiles)

FileNotFoundError: [Errno 2] No such file or directory: '/opt/workspace/id_df_a/part*.csv'

In [42]:
result_1 = requests.get(f'http://cluster-b:9300//send_data_to_alice')
print(result_1.content)


b'Everything was completed succesfully'


In [43]:

result_2 = requests.get(f'http://cluster-a:9200//send_data_to_bob')
print(result_2.content)

b'Everything was completed succesfully'
